# Feature Engineering for Student Success Prediction

This notebook creates features from all data sources for predicting student outcomes.

## Feature Categories
1. **Admissions Features** - Demographics, financial aid, exam type
2. **High School Exam Features** - Normalized scores from WASSCE, IB, A-Level, etc.
3. **Year 1 Academic Features** - GPA, courses, failures in first year
4. **Year 1-2 Academic Features** - Extended academic history
5. **Target Variables** - For each research question

In [46]:
import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Check if we are in the 'notebooks' directory and move up if so
if current_dir.endswith("notebooks"):
    os.chdir("..") # Move up one level to project root
    sys.path.append(os.getcwd()) # Add project root to python path

print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: /Users/user/coding/School/Ashesi/Semester-1/Machine-learning-&-data-science/final-project/project-claude


In [47]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Project paths
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


from src.data.loader import DataLoader
from src.data.exam_normalizer import (
    normalize_wassce_grade, normalize_ib_score, normalize_a_level_grade,
    normalize_french_bac_score, normalize_hsdiploma_grade, normalize_exam_score
)

# Load all datasets
print("Loading datasets...")
loader = DataLoader(data_path=str(DATA_DIR))
datasets = loader.load_all_datasets(verbose=True)

Loading datasets...
  ✓ Loaded application: 12,647 rows, 12,207 unique students
  ✓ Loaded cgpa: 24,648 rows, 3,718 unique students
  ✓ Loaded transcript: 115,421 rows, 3,684 unique students
  ✓ Loaded ajc: 143 rows, 134 unique students
  ✓ Loaded wassce: 1,274 rows, 1,264 unique students
  ✓ Loaded oa_level: 343 rows, 342 unique students
  ✓ Loaded hsdiploma: 51 rows, 51 unique students
  ✓ Loaded french: 23 rows, 23 unique students
  ✓ Loaded ib: 131 rows, 131 unique students
  ✓ Loaded other: 141 rows, 101 unique students


## 1. Admissions Features

Extract demographics, financial aid, and exam type information from application data.

In [48]:
def create_admissions_features(application_df):
    """Create admissions-based features from application data."""
    df = application_df.copy()
    features = pd.DataFrame()
    
    # Student identifier
    features['student_id'] = df['student_id']
    
    # Gender encoding
    features['gender_male'] = (df['Gender'] == 'M').astype(int)
    
    # International status (assume Country0 is Ghana/local)
    features['is_international'] = (df['Nationality'] != 'Country0').astype(int)
    
    # Offer type (for enrolled students)
    features['offer_enrolled'] = (df['Offer type'] == 'Enrolled').astype(int)
    
    # Intended major categories
    intended_major = df['Offer course name'].fillna('')
    features['intended_cs'] = intended_major.str.contains('Computer Science', case=False).astype(int)
    features['intended_engineering'] = intended_major.str.contains('Engineering', case=False).astype(int)
    features['intended_business'] = intended_major.str.contains('Business', case=False).astype(int)
    features['intended_mis'] = intended_major.str.contains('MIS|Information Systems', case=False).astype(int)
    
    # Financial aid need
    financial_aid_col = 'Extra question: Do you Need Financial Aid?'
    if financial_aid_col in df.columns:
        features['needs_financial_aid'] = (df[financial_aid_col] == 'Yes').astype(int)
    
    # Exam type category
    exam_type_col = 'Extra question: Type of Exam'
    if exam_type_col in df.columns:
        exam_type = df[exam_type_col].fillna('Unknown')
        features['exam_wassce'] = exam_type.str.contains('WASSCE', case=False, na=False).astype(int)
        features['exam_ib'] = exam_type.str.contains('IB|International Baccalaureate', case=False, na=False).astype(int)
        features['exam_alevel'] = exam_type.str.contains('A Level|IGCSE|KCSE', case=False, na=False).astype(int)
    
    # Previous application flag
    prev_app_col = 'Extra question: Have you applied to Ashesi before? If "yes" indicate the year.'
    if prev_app_col in df.columns:
        features['has_previous_application'] = (~df[prev_app_col].isin(['No', 'NaN', np.nan])).astype(int)
    
    # Disadvantaged background
    if 'Disadvantaged background' in df.columns:
        features['disadvantaged_background'] = df['Disadvantaged background'].notna().astype(int)
    
    return features.drop_duplicates('student_id')

# Create admissions features
admissions_features = create_admissions_features(datasets['application'])
print(f"Admissions features shape: {admissions_features.shape}")
print(f"\nFeature columns:")
for col in admissions_features.columns[1:]:
    print(f"  - {col}")
admissions_features.head()

Admissions features shape: (12207, 14)

Feature columns:
  - gender_male
  - is_international
  - offer_enrolled
  - intended_cs
  - intended_engineering
  - intended_business
  - intended_mis
  - needs_financial_aid
  - exam_wassce
  - exam_ib
  - exam_alevel
  - has_previous_application
  - disadvantaged_background


,student_id,gender_male,is_international,offer_enrolled,intended_cs,intended_engineering,intended_business,intended_mis,needs_financial_aid,exam_wassce,exam_ib,exam_alevel,has_previous_application,disadvantaged_background
0,Sd25fcbb18e84f890,1,0,0,1,0,0,0,1,1,0,0,1,0
1,Sfd5f545f824e3b45,1,0,0,0,0,0,1,0,0,0,1,0,0
2,Se4a2f9bcf28873f3,1,1,0,0,1,0,0,1,0,0,1,1,0
3,S2c5748435b37f518,1,0,0,0,1,0,0,1,0,0,1,0,0
4,Sa3d7b10c3d22ffe0,1,1,0,0,1,0,0,1,0,0,0,0,0


## 2. High School Exam Features

Normalize and extract features from all 6 exam types.

In [49]:
def create_wassce_features(wassce_df):
    """Create normalized features from WASSCE exam data."""
    df = wassce_df.copy()
    features = pd.DataFrame()
    features['student_id'] = df['student_id']
    features['exam_source'] = 'wassce'
    
    # Core subjects
    core_subjects = ['Mathematics', 'English Language', 'Integrated Science', 'Social Studies']
    
    for subj in core_subjects:
        if subj in df.columns:
            col_name = subj.lower().replace(' ', '_')
            features[f'hs_{col_name}'] = df[subj].apply(normalize_wassce_grade)
    
    # Elective math
    if 'Elective Math' in df.columns:
        features['hs_elective_math'] = df['Elective Math'].apply(normalize_wassce_grade)
        features['has_elective_math'] = df['Elective Math'].notna().astype(int)
    
    # Science subjects
    science_cols = ['Physics', 'Chemistry', 'Biology']
    science_scores = []
    for col in science_cols:
        if col in df.columns:
            features[f'hs_{col.lower()}'] = df[col].apply(normalize_wassce_grade)
            science_scores.append(f'hs_{col.lower()}')
    
    # Best science score
    if science_scores:
        features['hs_best_science'] = features[science_scores].max(axis=1)
    
    # Total aggregate (lower is better for WASSCE)
    if 'Total Aggregate' in df.columns:
        features['hs_total_aggregate'] = pd.to_numeric(df['Total Aggregate'], errors='coerce')
        # Invert aggregate for consistency (higher = better)
        features['hs_aggregate_score'] = 100 - features['hs_total_aggregate'] * 2  # Scale to ~0-100
        features['hs_aggregate_score'] = features['hs_aggregate_score'].clip(0, 100)
    
    return features

# Process WASSCE
if datasets['wassce'] is not None:
    wassce_features = create_wassce_features(datasets['wassce'])
    print(f"WASSCE features: {wassce_features.shape}")
    wassce_features.head()

WASSCE features: (1274, 14)


In [50]:
def create_ib_features(ib_df):
    """Create normalized features from IB exam data."""
    df = ib_df.copy()
    features = pd.DataFrame()
    features['student_id'] = df['student_id']
    features['exam_source'] = 'ib'
    
    # IB total points
    if 'Points' in df.columns:
        features['hs_total_points'] = pd.to_numeric(df['Points'], errors='coerce')
        # Normalize to 0-100 (IB max is typically 45)
        features['hs_aggregate_score'] = (features['hs_total_points'] / 45) * 100
    
    # Look for Math subjects
    math_cols = [col for col in df.columns if 'math' in col.lower()]
    for col in math_cols[:1]:  # Take first match
        features['hs_mathematics'] = df[col].apply(normalize_ib_score)
    
    # Look for English subjects
    eng_cols = [col for col in df.columns if 'english' in col.lower()]
    for col in eng_cols[:1]:
        features['hs_english_language'] = df[col].apply(normalize_ib_score)
    
    # Look for Science subjects
    science_cols = [col for col in df.columns if any(s in col.lower() for s in ['physics', 'chemistry', 'biology'])]
    if science_cols:
        science_scores = df[science_cols].applymap(normalize_ib_score)
        features['hs_best_science'] = science_scores.max(axis=1)
    
    return features

# Process IB
if datasets['ib'] is not None:
    ib_features = create_ib_features(datasets['ib'])
    print(f"IB features: {ib_features.shape}")

IB features: (131, 7)


In [51]:
def create_alevel_features(alevel_df):
    """Create normalized features from O/A Level exam data."""
    df = alevel_df.copy()
    features = pd.DataFrame()
    features['student_id'] = df['student_id']
    features['exam_source'] = 'a_level'
    
    # Points/aggregate
    if 'Points' in df.columns:
        features['hs_total_points'] = pd.to_numeric(df['Points'], errors='coerce')
        # Normalize (A-level points vary, rough scale)
        features['hs_aggregate_score'] = features['hs_total_points'].clip(0, 100)
    
    # Math
    math_cols = [col for col in df.columns if 'math' in col.lower()]
    for col in math_cols[:1]:
        features['hs_mathematics'] = df[col].apply(normalize_a_level_grade)
    
    # English
    eng_cols = [col for col in df.columns if 'english' in col.lower()]
    for col in eng_cols[:1]:
        features['hs_english_language'] = df[col].apply(normalize_a_level_grade)
    
    # Science
    science_cols = [col for col in df.columns if any(s in col.lower() for s in ['physics', 'chemistry', 'biology'])]
    if science_cols:
        for col in science_cols:
            features[f'hs_{col.lower().split()[0]}'] = df[col].apply(normalize_a_level_grade)
        features['hs_best_science'] = features[[f'hs_{col.lower().split()[0]}' for col in science_cols]].max(axis=1)
    
    return features

# Process O/A Level
if datasets['oa_level'] is not None:
    alevel_features = create_alevel_features(datasets['oa_level'])
    print(f"O/A Level features: {alevel_features.shape}")

O/A Level features: (343, 11)


In [52]:
def combine_exam_features(exam_feature_list):
    """Combine all exam features into a single DataFrame."""
    # Common columns to keep
    common_cols = ['student_id', 'exam_source', 'hs_mathematics', 'hs_english_language', 
                   'hs_best_science', 'hs_aggregate_score', 'has_elective_math']
    
    combined_list = []
    for df in exam_feature_list:
        if df is not None and len(df) > 0:
            # Add missing columns with NaN
            for col in common_cols:
                if col not in df.columns:
                    df[col] = np.nan
            combined_list.append(df[common_cols])
    
    if combined_list:
        combined = pd.concat(combined_list, ignore_index=True)
        return combined.drop_duplicates('student_id')
    return pd.DataFrame()

# Combine all exam features
exam_feature_list = []
if datasets['wassce'] is not None:
    exam_feature_list.append(create_wassce_features(datasets['wassce']))
if datasets['ib'] is not None:
    exam_feature_list.append(create_ib_features(datasets['ib']))
if datasets['oa_level'] is not None:
    exam_feature_list.append(create_alevel_features(datasets['oa_level']))

hs_exam_features = combine_exam_features(exam_feature_list)
print(f"\nCombined HS exam features: {hs_exam_features.shape}")
print(f"\nExam source distribution:")
print(hs_exam_features['exam_source'].value_counts())
hs_exam_features.head()


Combined HS exam features: (1717, 7)

Exam source distribution:
exam_source
wassce     1264
a_level     322
ib          131
Name: count, dtype: int64


,student_id,exam_source,hs_mathematics,hs_english_language,hs_best_science,hs_aggregate_score,has_elective_math
0,S7047a4e6df5e8cf5,wassce,70.0,80.0,NaN,78.0,0.0
1,S5e71a2543b6dae93,wassce,90.0,60.0,NaN,82.0,1.0
2,S1b0f4121c3b0de8a,wassce,90.0,90.0,90.0,86.0,1.0
3,Sd8609988972b7669,wassce,90.0,90.0,NaN,86.0,0.0
4,S81023cf42ee1bcb8,wassce,90.0,80.0,80.0,80.0,1.0


## 3. Year 1 Academic Features

Extract features from first-year academic performance (semesters 1-2).

In [53]:
import pandas as pd
import numpy as np

def create_year1_features(cgpa_df, transcript_df):
    """
    Create features strictly from Year 1 academic data (Semesters 1 & 2).
    correctly handling the 'Academic Year' vs 'Admission Year' logic.
    """
    
    # 1. Setup and Helpers
    # ---------------------------------------------------------
    cgpa = cgpa_df.copy()
    transcript = transcript_df.copy()

    def get_year_level(df):
        """Calculates Year Level (1, 2, 3...) based on Admission vs Academic Year"""
        # Split '2013-2014' -> take '2013' -> convert to int
        adm_start = df['Admission Year'].str.split('-').str[0].astype(int)
        acad_start = df['Academic Year'].str.split('-').str[0].astype(int)
        return acad_start - adm_start + 1

    cgpa['student_year_level'] = get_year_level(cgpa)
    transcript['student_year_level'] = get_year_level(transcript)

    # Keep only Year 1 rows
    cgpa_y1 = cgpa[cgpa['student_year_level'] == 1].copy()
    transcript_y1 = transcript[transcript['student_year_level'] == 1].copy()

    # 3. Clean and Sort Semester Data
    # ---------------------------------------------------------
    # Extract semester number (1 or 2)
    cgpa_y1['semester_num'] = cgpa_y1['Semester/Year'].str.extract(r'Semester\s*(\d+)').astype(int)
    transcript_y1['semester_num'] = transcript_y1['Semester/Year'].str.extract(r'Semester\s*(\d+)').astype(int)

    # Filter to ensure we only have Sem 1 and 2 (ignoring Summer/Sem 3 if present)
    target_sems = [1, 2]
    cgpa_y1 = cgpa_y1[cgpa_y1['semester_num'].isin(target_sems)]
    transcript_y1 = transcript_y1[transcript_y1['semester_num'].isin(target_sems)]

    cgpa_y1 = cgpa_y1.sort_values(['student_id', 'semester_num'])

    gpa_agg = cgpa_y1.groupby('student_id').agg({
        'GPA': ['mean', 'min', 'max', 'std'],
        'CGPA': ['last', 'min']  # 'last' is now safe because we sorted above
    })
    
    gpa_agg.columns = ['y1_gpa_mean', 'y1_gpa_min', 'y1_gpa_max', 'y1_gpa_std', 'y1_cgpa_end', 'y1_cgpa_min']
    features = gpa_agg.reset_index()

    # 5. Feature Engineering: GPA Trend (Sem 2 - Sem 1)
    # ---------------------------------------------------------
    # Pivot to get columns '1' and '2' for easy subtraction
    gpa_pivot = cgpa_y1.pivot(index='student_id', columns='semester_num', values='GPA')
    
    # If a student is missing Sem 2, the result will be NaN (handled later or left as is)
    gpa_trend = (gpa_pivot[2] - gpa_pivot[1]).rename('y1_gpa_trend')
    features = features.merge(gpa_trend, on='student_id', how='left')

    true_fails = ['E', 'F']           # 0.0 GPA
    risk_grades = ['D', 'D+', 'D-']   # 1.0 - 1.5 GPA (Passing but dangerous)
    
    transcript_y1['is_fail'] = transcript_y1['Grade'].isin(true_fails)
    transcript_y1['is_risk'] = transcript_y1['Grade'].isin(risk_grades)
    transcript_y1['is_a_grade'] = transcript_y1['Grade'].isin(['A', 'A-', 'A+'])

    course_agg = transcript_y1.groupby('student_id').agg({
        'Course Name': 'count',
        'is_fail': 'sum',
        'is_risk': 'sum',
        'is_a_grade': 'sum',
        'Grade point': 'mean'
    })

    course_agg.columns = ['y1_courses_taken', 'y1_fail_count', 'y1_risk_count', 'y1_a_count', 'y1_avg_grade_point']
    course_agg = course_agg.reset_index()

    # Calculate Rates
    course_agg['y1_fail_rate'] = course_agg['y1_fail_count'] / course_agg['y1_courses_taken']
    course_agg['y1_risk_rate'] = course_agg['y1_risk_count'] / course_agg['y1_courses_taken']
    course_agg['y1_a_rate'] = course_agg['y1_a_count'] / course_agg['y1_courses_taken']

    features = features.merge(course_agg, on='student_id', how='left')

    # 7. Final Flags
    # ---------------------------------------------------------
    # Flag if they ever dipped below 2.0 CGPA in Year 1
    features['y1_ever_probation'] = (features['y1_cgpa_min'] < 2.0).astype(int)

    # Fill NaN values for counts with 0 (e.g. if student had no transcript data but had GPA data)
    fill_cols = ['y1_fail_count', 'y1_risk_count', 'y1_a_count']
    features[fill_cols] = features[fill_cols].fillna(0)

    features['y1_gpa_std'] = features['y1_gpa_std'].fillna(0)
    
    # CASE B: GPA Trend
    # If missing Sem 2, we assume 'no change' (0) or distinct flag.
    # Safe option: Set trend to 0 (neutral).
    features['y1_gpa_trend'] = features['y1_gpa_trend'].fillna(0)
    
    # OPTIONAL: Create a flag so the model knows this data was missing
    # This helps the model learn "Students who miss Sem 2 are likely to dropout"
    features['y1_incomplete_data'] = (features['y1_courses_taken'] < 8).astype(int)

    return features

# Create Year 1 features
year1_features = create_year1_features(datasets['cgpa'], datasets['transcript'])
print(f"Year 1 features shape: {year1_features.shape}")
print(f"\nFeature columns:")
for col in year1_features.columns[1:]:
    print(f"  - {col}")
    
year1_features.head()

Year 1 features shape: (3218, 18)

Feature columns:
  - y1_gpa_mean
  - y1_gpa_min
  - y1_gpa_max
  - y1_gpa_std
  - y1_cgpa_end
  - y1_cgpa_min
  - y1_gpa_trend
  - y1_courses_taken
  - y1_fail_count
  - y1_risk_count
  - y1_a_count
  - y1_avg_grade_point
  - y1_fail_rate
  - y1_risk_rate
  - y1_a_rate
  - y1_ever_probation
  - y1_incomplete_data


,student_id,y1_gpa_mean,y1_gpa_min,y1_gpa_max,y1_gpa_std,y1_cgpa_end,y1_cgpa_min,y1_gpa_trend,y1_courses_taken,y1_fail_count,y1_risk_count,y1_a_count,y1_avg_grade_point,y1_fail_rate,y1_risk_rate,y1_a_rate,y1_ever_probation,y1_incomplete_data
0,S00039f6fd1b74390,3.88,3.88,3.88,0.000000,3.88,3.88,0.00,5.0,0.0,0.0,3.0,3.10,0.0,0.0,0.6,0,1
1,S0021eb5e8ac9bfec,3.75,3.75,3.75,0.000000,3.75,3.75,0.00,5.0,0.0,0.0,3.0,3.00,0.0,0.0,0.6,0,1
2,S0027200343737e85,3.55,3.22,3.88,0.466690,3.53,3.53,-0.66,10.0,0.0,0.0,4.0,3.15,0.0,0.0,0.4,0,0
3,S00351b9aa92d65d6,3.47,3.38,3.56,0.127279,3.47,3.38,0.18,10.0,0.0,0.0,4.0,3.15,0.0,0.0,0.4,0,0
4,S00364db68c4f235e,3.50,3.50,3.50,0.000000,3.50,3.50,0.00,5.0,0.0,0.0,2.0,2.80,0.0,0.0,0.4,0,1


In [54]:
datasets['cgpa'].head()

,Yeargroup,student_id,Admission Year,Program,Semester/Year,Academic Year,Student Status,Gender,Nationality,Application Category,GPA,CGPA
0,2017,Sb01f8b2a9888be6f,2013-2014,B.Sc - Management Information Systems,Semester 1,2015-2016,Active,Male,Country0,Unknown,3.63,3.26
1,2017,Sb01f8b2a9888be6f,2013-2014,B.Sc - Management Information Systems,Semester 2,2015-2016,Active,Male,Country0,Unknown,2.70,3.15
2,2017,S87f7615365ccf796,2013-2014,B.Sc - Computer Science,Semester 1,2015-2016,Active,Male,Country0,Unknown,3.60,3.66
3,2017,S87f7615365ccf796,2013-2014,B.Sc - Computer Science,Semester 2,2015-2016,Active,Male,Country0,Unknown,3.40,3.61
4,2018,Sfea019092a0d1158,2014-2015,B.Sc - Business Administration,Semester 1,2015-2016,Graduated,Female,Country1,Unknown,3.45,3.11


## 3b. Year 2 Academic Features

Extract features from second-year academic performance (Year 2, Semesters 1 & 2).

In [55]:
def create_year2_features(cgpa_df, transcript_df):
    """
    Create features strictly from Year 2 academic data.
    Similar to Year 1 features but for the second year.
    """

    # 1. Setup and Helpers
    cgpa = cgpa_df.copy()
    transcript = transcript_df.copy()

    def get_year_level(df):
        """Calculates Year Level (1, 2, 3...) based on Admission vs Academic Year"""
        adm_start = df['Admission Year'].str.split('-').str[0].astype(int)
        acad_start = df['Academic Year'].str.split('-').str[0].astype(int)
        return acad_start - adm_start + 1

    cgpa['student_year_level'] = get_year_level(cgpa)
    transcript['student_year_level'] = get_year_level(transcript)

    # Keep only Year 2 rows
    cgpa_y2 = cgpa[cgpa['student_year_level'] == 2].copy()
    transcript_y2 = transcript[transcript['student_year_level'] == 2].copy()

    if len(cgpa_y2) == 0:
        print("Warning: No Year 2 data found")
        return pd.DataFrame(columns=['student_id'])

    # 3. Clean and Sort Semester Data
    cgpa_y2['semester_num'] = cgpa_y2['Semester/Year'].str.extract(r'Semester\s*(\d+)').astype(float)
    transcript_y2['semester_num'] = transcript_y2['Semester/Year'].str.extract(r'Semester\s*(\d+)').astype(float)

    # Filter to ensure we only have Sem 1 and 2 (ignoring Summer/Sem 3 if present)
    target_sems = [1, 2]
    cgpa_y2 = cgpa_y2[cgpa_y2['semester_num'].isin(target_sems)]
    transcript_y2 = transcript_y2[transcript_y2['semester_num'].isin(target_sems)]

    cgpa_y2 = cgpa_y2.sort_values(['student_id', 'semester_num'])

    # 4. GPA Aggregation
    gpa_agg = cgpa_y2.groupby('student_id').agg({
        'GPA': ['mean', 'min', 'max', 'std'],
        'CGPA': ['last', 'min']
    })

    gpa_agg.columns = ['y2_gpa_mean', 'y2_gpa_min', 'y2_gpa_max', 'y2_gpa_std', 'y2_cgpa_end', 'y2_cgpa_min']
    features = gpa_agg.reset_index()

    # 5. GPA Trend (Sem 2 - Sem 1)
    gpa_pivot = cgpa_y2.pivot_table(index='student_id', columns='semester_num', values='GPA', aggfunc='first')

    if 1 in gpa_pivot.columns and 2 in gpa_pivot.columns:
        gpa_trend = (gpa_pivot[2] - gpa_pivot[1]).rename('y2_gpa_trend')
        features = features.merge(gpa_trend, on='student_id', how='left')
    else:
        features['y2_gpa_trend'] = 0

    # 6. Course-Level Features
    true_fails = ['E', 'F']
    risk_grades = ['D', 'D+', 'D-']

    transcript_y2['is_fail'] = transcript_y2['Grade'].isin(true_fails)
    transcript_y2['is_risk'] = transcript_y2['Grade'].isin(risk_grades)
    transcript_y2['is_a_grade'] = transcript_y2['Grade'].isin(['A', 'A-', 'A+'])

    course_agg = transcript_y2.groupby('student_id').agg({
        'Course Name': 'count',
        'is_fail': 'sum',
        'is_risk': 'sum',
        'is_a_grade': 'sum',
        'Grade point': 'mean'
    })

    course_agg.columns = ['y2_courses_taken', 'y2_fail_count', 'y2_risk_count', 'y2_a_count', 'y2_avg_grade_point']
    course_agg = course_agg.reset_index()

    # Calculate Rates
    course_agg['y2_fail_rate'] = course_agg['y2_fail_count'] / course_agg['y2_courses_taken']
    course_agg['y2_risk_rate'] = course_agg['y2_risk_count'] / course_agg['y2_courses_taken']
    course_agg['y2_a_rate'] = course_agg['y2_a_count'] / course_agg['y2_courses_taken']

    features = features.merge(course_agg, on='student_id', how='left')

    # 7. Final Flags
    features['y2_ever_probation'] = (features['y2_cgpa_min'] < 2.0).astype(int)

    # Fill NaN values
    fill_cols = ['y2_fail_count', 'y2_risk_count', 'y2_a_count']
    features[fill_cols] = features[fill_cols].fillna(0)
    features['y2_gpa_std'] = features['y2_gpa_std'].fillna(0)
    features['y2_gpa_trend'] = features['y2_gpa_trend'].fillna(0)

    features['y2_incomplete_data'] = (features['y2_courses_taken'] < 8).astype(int)

    return features

# Create Year 2 features
year2_features = create_year2_features(datasets['cgpa'], datasets['transcript'])
print(f"Year 2 features shape: {year2_features.shape}")
print(f"\nFeature columns:")
for col in year2_features.columns[1:]:
    print(f"  - {col}")

year2_features.head()

Year 2 features shape: (2854, 18)

Feature columns:
  - y2_gpa_mean
  - y2_gpa_min
  - y2_gpa_max
  - y2_gpa_std
  - y2_cgpa_end
  - y2_cgpa_min
  - y2_gpa_trend
  - y2_courses_taken
  - y2_fail_count
  - y2_risk_count
  - y2_a_count
  - y2_avg_grade_point
  - y2_fail_rate
  - y2_risk_rate
  - y2_a_rate
  - y2_ever_probation
  - y2_incomplete_data


,student_id,y2_gpa_mean,y2_gpa_min,y2_gpa_max,y2_gpa_std,y2_cgpa_end,y2_cgpa_min,y2_gpa_trend,y2_courses_taken,y2_fail_count,y2_risk_count,y2_a_count,y2_avg_grade_point,y2_fail_rate,y2_risk_rate,y2_a_rate,y2_ever_probation,y2_incomplete_data
0,S00039f6fd1b74390,3.805,3.78,3.83,0.035355,3.83,3.82,0.05,10.0,0.0,0.0,7.0,3.75,0.0,0.0,0.7,0,0
1,S0021eb5e8ac9bfec,3.000,2.83,3.17,0.240416,3.23,3.23,-0.34,10.0,0.0,1.0,0.0,3.05,0.0,0.1,0.0,0,0
2,S0027200343737e85,3.665,3.61,3.72,0.077782,3.60,3.60,-0.11,10.0,0.0,0.0,4.0,3.65,0.0,0.0,0.4,0,0
3,S002e2924edb73507,3.085,2.78,3.39,0.431335,2.97,2.97,-0.61,10.0,0.0,0.0,1.0,3.10,0.0,0.0,0.1,0,0
4,S00351b9aa92d65d6,3.670,3.56,3.78,0.155563,3.57,3.57,-0.22,10.0,0.0,0.0,7.0,3.70,0.0,0.0,0.7,0,0


## 4. Math Track Detection

Identify which math track each student started on (Calculus, Pre-Calculus, or College Algebra).

In [40]:
def detect_math_track(transcript_df):
    """Detect math track for each student from transcript data."""
    df = transcript_df.copy()
    
    # Define math track courses
    calculus_patterns = ['Calculus', 'MATH142']
    precalc_patterns = ['Pre-Calculus', 'Pre Calculus', 'PreCalculus', 'MATH141']
    algebra_patterns = ['College Algebra', 'Algebra', 'MATH140']
    
    def identify_track(course_name):
        course = str(course_name)
        for pattern in precalc_patterns:
            if pattern.lower() in course.lower():
                return 'precalculus'
        for pattern in calculus_patterns:
            if pattern.lower() in course.lower():
                return 'calculus'
        for pattern in algebra_patterns:
            if pattern.lower() in course.lower():
                return 'college_algebra'
        return None
    
    # Find math courses
    df['math_track_detected'] = df['Course Name'].apply(identify_track)
    
    # Get first math course per student (earliest semester)
    math_courses = df[df['math_track_detected'].notna()].copy()
    math_courses['semester_num'] = math_courses['Semester/Year'].str.extract(r'(\d+)').astype(float)
    
    # Get the first math track
    first_math = math_courses.sort_values('semester_num').groupby('student_id').first()
    first_math = first_math[['math_track_detected', 'Grade', 'Grade point']].reset_index()
    first_math.columns = ['student_id', 'math_track', 'first_math_grade', 'first_math_grade_point']
    
    # Encode math track
    track_encoding = {'calculus': 3, 'precalculus': 2, 'college_algebra': 1}
    first_math['math_track_encoded'] = first_math['math_track'].map(track_encoding)
    
    return first_math

# Detect math tracks
math_track_features = detect_math_track(datasets['transcript'])
print(f"Math track features: {math_track_features.shape}")
print(f"\nMath track distribution:")
print(math_track_features['math_track'].value_counts())
math_track_features.head()

Math track features: (3636, 5)

Math track distribution:
math_track
calculus           2035
precalculus        1288
college_algebra     313
Name: count, dtype: int64


,student_id,math_track,first_math_grade,first_math_grade_point,math_track_encoded
0,S00039f6fd1b74390,calculus,B+,3.5,3
1,S000901505ca1ec7f,calculus,A,4.0,3
2,S0021eb5e8ac9bfec,precalculus,B,3.0,2
3,S0027200343737e85,calculus,A,4.0,3
4,S002e2924edb73507,calculus,D+,1.5,3


## 5. Target Variable Creation

Create target variables for each research question.

In [41]:
def create_target_variables(cgpa_df, transcript_df, ajc_df):
    """Create all target variables for research questions with fixed logic."""
    
    # 1. Setup Copies
    cgpa = cgpa_df.copy()
    ajc = ajc_df.copy() if ajc_df is not None else pd.DataFrame()
    
    # 2. HELPER: Calculate True Year Level (Crucial for RQ1 and RQ9)
    def get_year_level(df):
        adm_start = df['Admission Year'].str.split('-').str[0].astype(int)
        acad_start = df['Academic Year'].str.split('-').str[0].astype(int)
        return acad_start - adm_start + 1

    cgpa['student_year_level'] = get_year_level(cgpa)
    
    # Extract semester number for sorting (1 or 2)
    cgpa['semester_int'] = cgpa['Semester/Year'].str.extract(r'Semester\s*(\d+)').astype(float)

    # 3. CRITICAL: Sort Data Chronologically
    # This ensures .last() actually gets the final CGPA
    cgpa = cgpa.sort_values(['student_id', 'student_year_level', 'semester_int'])

    # Initialize Targets DataFrame
    all_students = set(cgpa['student_id'].unique())
    targets = pd.DataFrame({'student_id': list(all_students)})
    
    # ---------------------------------------------------------
    # RQ1: First year struggle (CGPA < 2.0 in Year 1 ONLY)
    # ---------------------------------------------------------
    # Filter strictly for Year Level 1
    y1_data = cgpa[cgpa['student_year_level'] == 1]
    
    # Check minimum CGPA during Year 1
    y1_min_cgpa = y1_data.groupby('student_id')['CGPA'].min()
    
    targets = targets.merge(
        (y1_min_cgpa < 2.0).rename('target_y1_struggle').reset_index(),
        on='student_id', how='left'
    )
    # Fill NaN with 0 (If no Year 1 data, they didn't struggle in Year 1)
    targets['target_y1_struggle'] = targets['target_y1_struggle'].fillna(0).astype(int)
    
    # ---------------------------------------------------------
    # RQ2: AJC case
    # ---------------------------------------------------------
    if len(ajc) > 0:
        ajc_students = set(ajc['student_id'].unique())
        targets['target_ajc_case'] = targets['student_id'].isin(ajc_students).astype(int)
        
        # Check verdict column existence just in case
        if 'Verdict' in ajc.columns:
            guilty_students = set(ajc[ajc['Verdict'] == 'Guilty']['student_id'].unique())
            targets['target_ajc_guilty'] = targets['student_id'].isin(guilty_students).astype(int)
        else:
            targets['target_ajc_guilty'] = 0
    else:
        targets['target_ajc_case'] = 0
        targets['target_ajc_guilty'] = 0
    
    # ---------------------------------------------------------
    # RQ3/5 & RQ7: Final CGPA (Success/Continuous)
    # ---------------------------------------------------------
    # Because we sorted in step 3, .last() is now safe
    final_cgpa = cgpa.groupby('student_id')['CGPA'].last()
    
    # Continuous Target
    targets = targets.merge(
        final_cgpa.rename('target_final_cgpa').reset_index(),
        on='student_id', how='left'
    )
    
    # Binary Target (Graduation CGPA >= 3.0)
    targets['target_major_success'] = (targets['target_final_cgpa'] >= 3.0).astype(int)
    
    # ---------------------------------------------------------
    # RQ4/6: Ever on probation
    # ---------------------------------------------------------
    ever_probation = cgpa.groupby('student_id')['CGPA'].min() < 2.0
    targets = targets.merge(
        ever_probation.rename('target_ever_probation').reset_index(),
        on='student_id', how='left'
    )
    targets['target_ever_probation'] = targets['target_ever_probation'].fillna(0).astype(int)
    
    # ---------------------------------------------------------
    # RQ9: Extended graduation (> 4 Years / 8 Semesters)
    # ---------------------------------------------------------
    # Logic: Did they register for a 5th year (Year Level > 4)?
    max_year_level = cgpa.groupby('student_id')['student_year_level'].max()
    
    targets = targets.merge(
        (max_year_level > 4).rename('target_extended_graduation').reset_index(),
        on='student_id', how='left'
    )
    targets['target_extended_graduation'] = targets['target_extended_graduation'].fillna(0).astype(int)
    
    return targets

# Create targets
target_variables = create_target_variables(datasets['cgpa'], datasets['transcript'], datasets['ajc'])
print(f"Target variables shape: {target_variables.shape}")
print(f"\nTarget variable distributions:")
for col in target_variables.columns[1:]:
    if col.startswith('target_') and col != 'target_final_cgpa':
        rate = target_variables[col].mean() * 100
        print(f"  {col}: {rate:.1f}% positive")
target_variables.head()

Target variables shape: (3718, 8)

Target variable distributions:
  target_y1_struggle: 9.1% positive
  target_ajc_case: 3.6% positive
  target_ajc_guilty: 3.0% positive
  target_major_success: 55.5% positive
  target_ever_probation: 11.4% positive
  target_extended_graduation: 7.0% positive


,student_id,target_y1_struggle,target_ajc_case,target_ajc_guilty,target_final_cgpa,target_major_success,target_ever_probation,target_extended_graduation
0,S089e85c637908d0d,0,0,0,3.68,1,0,0
1,S109c6005789867d8,0,0,0,3.39,1,0,0
2,Scad6730d086a23f0,0,0,0,2.46,0,0,0
3,S917b929eab83a362,0,0,0,2.91,0,0,0
4,S4900bb2ad4c106cb,0,0,0,2.61,0,0,0


## 6. Combine All Features

Merge all feature sets into final datasets for modeling.

In [42]:
def create_modeling_dataset(admissions, hs_exam, year1, year2, math_track, targets):
    """Combine all features into modeling datasets."""
    
    # Start with targets
    full_data = targets.copy()
    
    # Merge admissions features
    full_data = full_data.merge(admissions, on='student_id', how='left')
    
    # Merge HS exam features
    full_data = full_data.merge(hs_exam, on='student_id', how='left')
    
    # Merge Year 1 features
    full_data = full_data.merge(year1, on='student_id', how='left')
    
    # Merge Year 2 features
    full_data = full_data.merge(year2, on='student_id', how='left')
    
    # Merge math track features
    full_data = full_data.merge(math_track, on='student_id', how='left')
    
    return full_data

# Create full dataset
full_features = create_modeling_dataset(
    admissions_features, 
    hs_exam_features, 
    year1_features,
    year2_features,
    math_track_features, 
    target_variables
)

print(f"Full features dataset: {full_features.shape}")
print(f"\nColumns ({len(full_features.columns)}):")
for i, col in enumerate(full_features.columns):
    dtype = full_features[col].dtype
    missing = full_features[col].isna().sum()
    print(f"  {i+1}. {col} ({dtype}, {missing} missing)")

Full features dataset: (3718, 65)

Columns (65):
  1. student_id (object, 0 missing)
  2. target_y1_struggle (int64, 0 missing)
  3. target_ajc_case (int64, 0 missing)
  4. target_ajc_guilty (int64, 0 missing)
  5. target_final_cgpa (float64, 0 missing)
  6. target_major_success (int64, 0 missing)
  7. target_ever_probation (int64, 0 missing)
  8. target_extended_graduation (int64, 0 missing)
  9. gender_male (float64, 1539 missing)
  10. is_international (float64, 1539 missing)
  11. offer_enrolled (float64, 1539 missing)
  12. intended_cs (float64, 1539 missing)
  13. intended_engineering (float64, 1539 missing)
  14. intended_business (float64, 1539 missing)
  15. intended_mis (float64, 1539 missing)
  16. needs_financial_aid (float64, 1539 missing)
  17. exam_wassce (float64, 1539 missing)
  18. exam_ib (float64, 1539 missing)
  19. exam_alevel (float64, 1539 missing)
  20. has_previous_application (float64, 1539 missing)
  21. disadvantaged_background (float64, 1539 missing)
  22.

In [43]:
# Summary statistics
print("\nDataset Summary:")
print(f"Total students: {len(full_features)}")
print(f"Students with HS exam data: {full_features['exam_source'].notna().sum()}")
print(f"Students with Year 1 data: {full_features['y1_gpa_mean'].notna().sum()}")
print(f"Students with Year 2 data: {full_features['y2_gpa_mean'].notna().sum()}")
print(f"Students with math track: {full_features['math_track'].notna().sum()}")

# Check class balance
print("\nTarget Variable Balance:")
for col in ['target_y1_struggle', 'target_ajc_case', 'target_major_success', 
            'target_ever_probation', 'target_extended_graduation']:
    positive = full_features[col].sum()
    total = full_features[col].notna().sum()
    print(f"  {col}: {positive}/{total} ({positive/total*100:.1f}%)")


Dataset Summary:
Total students: 3718
Students with HS exam data: 1717
Students with Year 1 data: 3218
Students with Year 2 data: 2854
Students with math track: 3636

Target Variable Balance:
  target_y1_struggle: 339/3718 (9.1%)
  target_ajc_case: 134/3718 (3.6%)
  target_major_success: 2065/3718 (55.5%)
  target_ever_probation: 425/3718 (11.4%)
  target_extended_graduation: 262/3718 (7.0%)


In [44]:
# Save processed datasets
print("\nSaving processed datasets...")

# Full feature set
full_features.to_csv(PROCESSED_DIR / 'full_features.csv', index=False)
print(f"  Saved: full_features.csv ({len(full_features)} rows)")

# Admissions-only features (for RQ1, RQ2)
admissions_only = full_features[['student_id'] + 
    [col for col in full_features.columns if col.startswith('target_')] +
    [col for col in admissions_features.columns if col != 'student_id'] +
    [col for col in hs_exam_features.columns if col != 'student_id']
].drop_duplicates('student_id')
admissions_only.to_csv(PROCESSED_DIR / 'admissions_features.csv', index=False)
print(f"  Saved: admissions_features.csv ({len(admissions_only)} rows)")

# Year 1 features
year1_features.to_csv(PROCESSED_DIR / 'year1_features.csv', index=False)
print(f"  Saved: year1_features.csv ({len(year1_features)} rows)")

# Year 2 features
year2_features.to_csv(PROCESSED_DIR / 'year2_features.csv', index=False)
print(f"  Saved: year2_features.csv ({len(year2_features)} rows)")

# Math track features
math_track_features.to_csv(PROCESSED_DIR / 'math_track_features.csv', index=False)
print(f"  Saved: math_track_features.csv ({len(math_track_features)} rows)")

# Target variables
target_variables.to_csv(PROCESSED_DIR / 'targets.csv', index=False)
print(f"  Saved: targets.csv ({len(target_variables)} rows)")

print(f"\nAll files saved to: {PROCESSED_DIR}")


Saving processed datasets...
  Saved: full_features.csv (3718 rows)
  Saved: admissions_features.csv (3718 rows)
  Saved: year1_features.csv (3218 rows)
  Saved: year2_features.csv (2854 rows)
  Saved: math_track_features.csv (3636 rows)
  Saved: targets.csv (3718 rows)

All files saved to: /Users/user/coding/School/Ashesi/Semester-1/Machine-learning-&-data-science/final-project/project-claude/data/processed


In [45]:
print("\n" + "="*60)
print(" FEATURE ENGINEERING COMPLETE ")
print("="*60)
print(f"\nFeature sets created:")
print(f"  1. full_features.csv - All features combined")
print(f"  2. admissions_features.csv - Admissions + HS exam only")
print(f"  3. year1_features.csv - Year 1 academic features")
print(f"  4. math_track_features.csv - Math track detection")
print(f"  5. targets.csv - All target variables")
print(f"\nNext notebook: 04_unsupervised_learning.ipynb")


 FEATURE ENGINEERING COMPLETE 

Feature sets created:
  1. full_features.csv - All features combined
  2. admissions_features.csv - Admissions + HS exam only
  3. year1_features.csv - Year 1 academic features
  4. math_track_features.csv - Math track detection
  5. targets.csv - All target variables

Next notebook: 04_unsupervised_learning.ipynb
